<h1 align='center'>Final assignment of 'Management and Analysis of Physics Datasets'</h1>

<h2 align='center'>Part 1: Distributed computing: Spark</h2>

<h3 align='left'>University of Padua - Physics of Data</h3>

<h4 align='right'>Prof. Jacopo Pazzini</h4>
<img align='center' src='https://www.unidformazione.com/wp-content/uploads/2018/04/unipd-universita-di-padova.png' alt='Drawing' style='width:400px;'/>

<!-- <h5 align='center'>Group members</h5> -->

**Name** | **ID number** | **mail**@studenti.unipd.it
:-:|:-:|-:
Chiara Maccani | 2027591 | chiara.maccani
Samuele Piccinelli | 2027650 | samuele.piccinelli
Tommaso Stentella | 2027586 | tommaso.stentella
Cristina Venturini | 2022461 | cristina.venturini.5

***

In [1]:
# - import libraries and useful dependencies

import numpy as np
import math as mh
import uproot
import requests
import ssl

import findspark
findspark.init('/usr/local/spark')

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib.lines import Line2D

## 1. Setting up Spark session

The number of cores assigned to each executor is configurable. When `spark.executor.cores` is explicitly set, multiple executors from the same application may be launched on the same worker if the worker has enough cores and memory. Otherwise, each executor grabs all the cores available on the worker by default, in which case only one executor per application may be launched on each worker during one single schedule iteration.

In [3]:
%%capture

spark = SparkSession.builder \
        .master('spark://10.67.22.59:7077') \
        .appName('first') \
        .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:2.7.3,ch.cern.sparkmeasure:spark-measure_2.12:0.17') \
        .config('spark.executor.cores', '2') \
        .config('spark.executor.memory','6g') \
        .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.nio.file.FileSystemException: /root/.ivy2/jars/com.fasterxml.jackson.core_jackson-databind-2.9.9.jar -> /tmp/spark-068be7b0-e00f-47ff-8006-64f1808ffc4f/userFiles-b1dd9d6b-8e76-4b61-9310-987629024498/com.fasterxml.jackson.core_jackson-databind-2.9.9.jar: No space left on device
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:91)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixCopyFile.copyFile(UnixCopyFile.java:253)
	at sun.nio.fs.UnixCopyFile.copy(UnixCopyFile.java:581)
	at sun.nio.fs.UnixFileSystemProvider.copy(UnixFileSystemProvider.java:253)
	at java.nio.file.Files.copy(Files.java:1274)
	at org.apache.spark.util.Utils$.copyRecursive(Utils.scala:726)
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:697)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:771)
	at org.apache.spark.util.Utils$.fetchFile(Util

In [3]:
# spark

In [4]:
sc = spark.sparkContext

NameError: name 'spark' is not defined

## 2. Read `.root` files stored remotely from S3 bucket

In [ ]:
import os
import configparser

config = configparser.ConfigParser()
config.read(os.path.expanduser('~/.s3cfg'))
access_id = config.get('aws_profile', 'access_key') 
access_key = config.get('aws_profile', 'secret_key')

In [ ]:
hadoop_conf=sc._jsc.hadoopConfiguration()

hadoop_conf.set('fs.s3n.impl', 'org.apache.hadoop.fs.s3native.NativeS3FileSystem')
hadoop_conf.set('fs.s3n.awsAccessKeyId', access_id)
hadoop_conf.set('fs.s3n.awsSecretAccessKey', access_key)

## 3. Path and dataset

In [ ]:
%%capture

buck = 'https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/LCPmodB-Y3_CMS_FCNC/'

listDir = requests.get(buck, verify=False).text.split('\n')
# listDir

In [ ]:
def returnDir(string):
    return [filename for filename in listDir if filename.startswith(string)]

In [ ]:
# - Data
dataDirs = [buck + file for file in returnDir('SingleMuon')]

# - MC Signal
signalMCDirs = [buck + file for file in returnDir('TT_FCNC')]

# - MC backgrounds
listBkgDir = ['ST_', 'TTTT_Tune', 'TTTo2L2Nu', 'TTToHadronic', 'TTToSemiLeptonic', 'TTWJetsToLNu', 'TTZToLLNuNu',
              'WGToLNuG', 'WJetsToLNu', 'WWTo2L2Nu', 'WWW', 'WWZ', 'WZG', 'WZTo1L1Nu2Q', 'WZTo2L2Q', 'WZTo3LNu',
              'WZZ', 'WmWmJJ', 'WminusH', 'WpWpJJ', 'ZG', 'ZZ', 'tZq', 'DYJetsToLL_M', 'QCD']

bkgMCDirs = dict(list(zip(listBkgDir, map(returnDir, listBkgDir))))

for j in listBkgDir:
    for i in range(len(bkgMCDirs[j])):
        bkgMCDirs[j][i] = buck + bkgMCDirs[j][i]

## 4. Read and load data

In [5]:
# - custom data loader
def readRoot(file_name):
    
    if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
        ssl._create_default_https_context = ssl._create_unverified_context
    
    var = ['iSkim', 'event', 'eventWeightLumi', 'nMuon','Muon_pfRelIso03_all', 'Muon_pt', 'Muon_eta',
               'Muon_phi', 'Muon_charge', 'nElectron','Electron_pfRelIso03_all','Electron_pt', 'Electron_phi',
               'Electron_charge','nCleanJet','nBJet','MET_pt', 'MET_phi']

    pro = uproot.open(file_name + ':Events').arrays(var, library='np')
    nEvent = len(pro['event'])
    custom_data = [0 for _ in range(nEvent)]

    for i in range(nEvent):
        event = dict(list(zip(var, [pro[v][i].tolist() for v in var])))
        custom_data[i] = event
        
    return custom_data

In [ ]:
# - load data
dfData = sc.parallelize(dataDirs).flatMap(readRoot).toDF()

In [ ]:
# - load MC backgrounds
dfMCBkg = {}
for key, value in bkgMCDirs.items():
    dfMCBkg[key] = sc.parallelize(value).flatMap(readRoot).toDF()

In [ ]:
# - load MC signal
dfMCSig = sc.parallelize(signalMCDirs).flatMap(readRoot).toDF()

## Measuring performance

In [6]:
# dicto =  list(map(returnDir, listBkgDir))
# flat_list = [item for sublist in dicto for item in sublist]
# returnDir('tZq')
listo = ['/data/FCNC/' + diro for diro in returnDir('tZq')]

size = sum(os.path.getsize(f) for f in listo)/1e9
size
size#*1e3/56

NameError: name 'returnDir' is not defined

In [ ]:
trial_dir = bkgMCDirs['tZq']

In [ ]:
from sparkmeasure import StageMetrics
stagemetrics = StageMetrics(spark)

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

#@register_line_cell_magic
def sparkmeasure(df, idx, ex = 2):#line, cell=None):
    #val = cell if cell is not None else line
    stagemetrics.begin()
    #eval(val)
    df.count()
    stagemetrics.end()
    
    df_out = stagemetrics.create_stagemetrics_DF("PerfStageMetrics")
    stagemetrics.save_data(df_out, "./perf_test/stagemetrics_{}_{}".format(ex, idx))

In [ ]:
part = [4,12,24,36,48,56,64]

In [ ]:
trialdf = [0,0,0,0,0,0,0]
for i in range(len(part)): 
    trialdf[i] = sc.parallelize(trial_dir, part[i]).flatMap(readRoot).toDF()

In [ ]:
#sparkmeasure(trialdf, 4)

In [ ]:
sparkmeasure(trialdf[0], part[0])

In [ ]:
sparkmeasure(trialdf[1], part[1])

In [ ]:
sparkmeasure(trialdf[2], part[2])

In [ ]:
sparkmeasure(trialdf[3], part[3])

In [ ]:
sparkmeasure(trialdf[4], part[4])

In [ ]:
sparkmeasure(trialdf[5], part[5])

In [ ]:
sparkmeasure(trialdf[6], part[6])

In [ ]:
%%sparkmeasure
trialdf.count()

## 5. Skimming and HLF creation

In [ ]:
# dfData.count()
# dfData.printSchema()
# dfData.persist()

### 5.1 First level filter

In [ ]:
import pyspark.sql.functions as pf
import pyspark.sql.types as pt

arr = pt.ArrayType
maskMu = pf.udf(lambda iso, pt: [True if i < .15 and p > 15 else False for i,p in zip(iso,pt)], arr(pt.BooleanType()))
maskEl = pf.udf(lambda iso: [True if i < .15 else False for i in iso], arr(pt.BooleanType()))
apply_mask_long = pf.udf(lambda x, mask: [i for i,m in zip(x,mask) if m], arr(pt.LongType()))
apply_mask_doub = pf.udf(lambda x, mask: [i for i,m in zip(x,mask) if m], arr(pt.DoubleType()))
count_col = pf.udf(lambda m: abs(sum(m)), pt.IntegerType())
mass_mu = pf.lit(.1057)

In [ ]:
def dfSkimmed(df):
    fdf = df.filter('iSkim == 3')\
    .withColumn('maskMu', maskMu('Muon_pfRelIso03_all', 'Muon_pt'))\
    .withColumn('nGoodMu', count_col('maskMu'))\
    .withColumn('maskEl', maskEl('Electron_pfRelIso03_all'))\
    .drop('Muon_pfRelIso03_all', 'Electron_pfRelIso03_all')\
    .withColumn('nGoodEl', count_col('maskEl'))\
    .filter('(nGoodMu == 3) AND (nGoodEl == 0)')\
    .drop('nGoodMu', 'nGoodEl')\
    .withColumn('SumCharge', count_col(apply_mask_long('Muon_charge', 'maskMu')))\
    .filter('SumCharge != 3')\
    .drop('SumCharge')\
    .withColumn('sMuon_pt', apply_mask_doub('Muon_pt', 'maskMu'))\
    .withColumn('sMuon_eta', apply_mask_doub('Muon_eta', 'maskMu'))\
    .withColumn('sMuon_phi', apply_mask_doub('Muon_phi', 'maskMu'))\
    .withColumn('sMuon_charge', apply_mask_long('Muon_charge', 'maskMu'))\
    .withColumn('sMuon_mass', pf.array(mass_mu, mass_mu, mass_mu))\
    .drop('Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_charge')\
    .filter('(nCleanJet >= 2) AND (nBJet >= 1)')\
    .drop('nCleanJet', 'nBJet')
    
    return fdf

In [ ]:
fdata = dfSkimmed(dfData)
fsignal = dfSkimmed(dfMCSig)

fbkg = {}
for key, value in dfMCBkg.items():
    fbkg[key] = dfSkimmed(value)

### 5.2 Second level filter: $\Delta R$, $\Delta\phi$ and $m_{inv}$

In [ ]:
def DeltaR(eta1, eta2, phi1, phi2):
    return mh.sqrt((eta1-eta2)**2 + (phi1-phi2)**2)

@pf.udf(arr(pt.IntegerType()))
def find_idx(charge, phi, eta):
   
    idx = [-1]*len(charge)
    
    if(charge[0]==charge[1] and charge[1]!=charge[2]):
        idx[0] = 2
        dr_1 = DeltaR(eta[1], eta[2], phi[1], phi[2])
        dr_2 = DeltaR(eta[0], eta[2], phi[0], phi[2])
        
        if(dr_1 < dr_2): idx[1], idx[2] = 1, 0
        else: idx[1], idx[2] = 0, 1
        
    if(charge[0]!=charge[1] and charge[1]!=charge[2]): 
        idx[0] = 1
        dr_1 = DeltaR(eta[1], eta[2], phi[1], phi[2])
        dr_2 = DeltaR(eta[0], eta[1], phi[0], phi[1])
        
        if(dr_1 < dr_2): idx[1], idx[2] = 2, 0
        else: idx[1], idx[2] = 2, 0
    
    if(charge[0]!=charge[1] and charge[1]==charge[2]):
        idx[0] = 0
        dr_1 = DeltaR(eta[0], eta[2], phi[0], phi[2])
        dr_2 = DeltaR(eta[0], eta[1], phi[0], phi[1])
        
        if(dr_1 < dr_2): idx[1], idx[2] = 2, 1
        else: idx[1], idx[2] = 1, 2   
        
    return idx

In [ ]:
dR = pf.udf(lambda eta1, eta2, phi1, phi2: DeltaR(eta1, eta2, phi1, phi2), pt.DoubleType())

return_value_long = pf.udf(lambda x,idx,val: x[idx[val]], pt.LongType())
return_value_doub = pf.udf(lambda x,idx,val: x[idx[val]], pt.DoubleType())

In [ ]:
def invMass(pt, eta, phi, mass):
    
    size = len(pt)
    assert(len(eta) == size and len(phi) == size and len(mass) == size)
  
    x_sum, y_sum, z_sum, e_sum = 0., 0., 0., 0.
  
    for i in range(size):
        # Convert to (e, x, y, z) coordinate system and update sums
        x = pt[i] * mh.cos(phi[i])
        x_sum += x
        y = pt[i] * mh.sin(phi[i])
        y_sum += y
        z = pt[i] * mh.sinh(eta[i])
        z_sum += z
        e = mh.sqrt(x * x + y * y + z * z + mass[i] * mass[i])
        e_sum += e
        
    return mh.sqrt(e_sum * e_sum - x_sum * x_sum - y_sum * y_sum - z_sum * z_sum)

In [ ]:
invMass3 = pf.udf(lambda (pt, eta, phi, mass): invMass(pt, eta, phi, mass), pt.DoubleType())
invMass2 = pf.udf(lambda (pt1, pt2, eta1, eta2, phi1, phi2, mass1, mass2):
                 invMass([pt1, pt2], [eta1, eta2], [phi1, phi2], [mass1, mass2]), pt.DoubleType())

In [ ]:
def dfHLF(df):
    fdf = df.withColumn('mu_idx', find_idx('sMuon_charge', 'sMuon_phi', 'sMuon_eta'))\
    .withColumn('mu_pt0', return_value_doub('sMuon_pt', 'mu_idx', pf.lit(0)))\
    .withColumn('mu_pt1', return_value_doub('sMuon_pt', 'mu_idx', pf.lit(1)))\
    .withColumn('mu_pt2', return_value_doub('sMuon_pt', 'mu_idx', pf.lit(2)))\
    .withColumn('mu_eta0', return_value_doub('sMuon_eta', 'mu_idx', pf.lit(0)))\
    .withColumn('mu_eta1', return_value_doub('sMuon_eta', 'mu_idx', pf.lit(1)))\
    .withColumn('mu_eta2', return_value_doub('sMuon_eta', 'mu_idx', pf.lit(2)))\
    .withColumn('mu_phi0', return_value_doub('sMuon_phi', 'mu_idx', pf.lit(0)))\
    .withColumn('mu_phi1', return_value_doub('sMuon_phi', 'mu_idx', pf.lit(1)))\
    .withColumn('mu_phi2', return_value_doub('sMuon_phi', 'mu_idx', pf.lit(2)))\
    .withColumn('mu_q0', return_value_long('sMuon_charge', 'mu_idx', pf.lit(0)))\
    .withColumn('mu_q1', return_value_long('sMuon_charge', 'mu_idx', pf.lit(1)))\
    .withColumn('mu_q2', return_value_long('sMuon_charge', 'mu_idx', pf.lit(2)))\
    .withColumn('mu_mass0', pf.lit(0.1057))\
    .withColumn('mu_mass1', pf.lit(0.1057))\
    .withColumn('mu_mass2', pf.lit(0.1057))\
    .withColumn('inv_m3', invMass3('sMuon_pt', 'sMuon_eta', 'sMuon_phi', 'sMuon_mass'))\
    .withColumn('inv_m01', invMass2('mu_pt0', 'mu_pt1', 'mu_eta0', 'mu_eta1', 'mu_phi0', 'mu_phi1', 'mu_mass0', 'mu_mass1'))\
    .withColumn('inv_m02', invMass2('mu_pt0', 'mu_pt2', 'mu_eta0', 'mu_eta2', 'mu_phi0', 'mu_phi2', 'mu_mass0', 'mu_mass2'))\
    .withColumn('inv_m12', invMass2('mu_pt1', 'mu_pt2', 'mu_eta1', 'mu_eta2', 'mu_phi1', 'mu_phi2', 'mu_mass1', 'mu_mass2'))\
    .filter('(inv_m01 > 15) AND (inv_m02 > 15) AND (inv_m12 > 15)')\
    .filter('(abs(inv_m01 - 91.2) > 10) AND (abs(inv_m02 - 91.2) > 10)')\
    .withColumn('dR01', dR('mu_eta0', 'mu_eta1', 'mu_phi0', 'mu_phi1'))\
    .withColumn('dR02', dR('mu_eta0', 'mu_eta2', 'mu_phi0', 'mu_phi2'))\
    .withColumn('dPhi0', pf.abs(pf.col('MET_phi') - pf.col('mu_phi0')))\
    .withColumn('dPhi1', pf.abs(pf.col('MET_phi') - pf.col('mu_phi1')))\
    .withColumn('dPhi2', pf.abs(pf.col('MET_phi') - pf.col('mu_phi2')))
    
    return fdf

In [ ]:
data = dfHLF(fdata)
signal = dfHLF(fsignal)

bkg = {}
for key, value in fbkg.items():
    bkg[key] = dfHLF(value)

## 6. Save the datasets as parquet files

In [ ]:
DATASETS_PATH = "./dataMAPD"
%time features.write.parquet(DATASETS_PATH+"datasets.parquet", mode="overwrite")

In [ ]:
data = spark.read \
            .format("parquet") \
            .load("hdfs://hadalytic/project/ML/data/swan/datasets.parquet")

events = data.count()
print("There are {} events".format(events))

## 7. Final histogram

In [ ]:
def histo(variable, weight, range_, bins):
    edges = np.array([i*(range_[1]-range_[0])/bins for i in range(0,bins+1)])
    edges = edges+range_[0]
    
    lim_couples = [[edges[i], edges[i+1]] for i in range(len(edges)-1)]
    
    def filtr(limits):
        if type(variable) != list:
            if (variable >= limits[0]) and (variable < limits[1]): return 1
            else: return 0
        else:
            lim0 = np.full(len(variable),limits[0])
            lim1 = np.full(len(variable),limits[1])
            mask = np.logical_and((variable >= lim0), (variable < lim1))
            if np.any(mask): return np.sum(mask)
            else: return 0
    
    counts = np.array(list(map(filtr, lim_couples))) * weight
    
    return edges, counts

In [ ]:
def plot_histogram(variable, interval, nbins):
    
    ### HISTOGRAM BINS COUNTING ###
    
    data_bins, data_counts = (
            data.select([variable,'eventWeightLumi']).rdd
                .map(lambda x: histo(x[variable], x['eventWeightLumi'], range_=interval, bins=nbins))
                .reduce(lambda x,y: (x[0], x[1]+y[1])))
    data_bin_centers = data_bins[:-1] + np.diff(data_bins)/2                                                                                   

    SigMC_bins, SigMC_counts = (
            signal.select([variable,'eventWeightLumi']).rdd
                  .map(lambda x: histo(x[variable], x['eventWeightLumi'], range_=interval, bins=nbins))
                  .reduce(lambda x,y: (x[0], x[1]+y[1])))
    SigMC_bin_centers = SigMC_bins[:-1] + np.diff(SigMC_bins)/2  

    #MCbkg_hist_results = {}
    bkgMC_bin_centers = []
    bkgMC_counts = []
    bkgMC_labels = dfMCBkg.keys()

    for value in bkg.values():
        #MCbkg_hist_results[key] = {}
        bkg_rdd = value.select([variable,'eventWeightLumi']).rdd
        if bkg_rdd.isEmpty():
            counts = np.full(nbins, 0)
            bins = np.array([i*(interval[1]-interval[0])/nbins for i in range(0, nbins+1)])
            bins = bins + interval[0]
        else:
            bins, counts = (
                bkg_rdd
                    .map(lambda x: histo(x[variable], x['eventWeightLumi'], range_=interval, bins=nbins))
                    .reduce(lambda x,y: (x[0], x[1]+y[1])))
        bin_centers = bins[:-1] + np.diff(bins)/2

        bkgMC_bin_centers.append(bin_centers)                                                                                             
        bkgMC_counts.append(counts)
        
    ### SIGNAL RESCALING ###
        
    bkg_values, bkg_bins, _ = plt.hist(bkgMC_bin_centers, weights=bkgMC_counts, bins=bins,
                                       stacked=True, histtype= 'barstacked')
    areaBkg = sum(np.diff(bkg_bins)*bkg_values)
    sig_values, sig_bins, _ = plt.hist(SigMC_bin_centers, weights=SigMC_counts, bins=SigMC_bins)
    areaSig = sum(np.diff(sig_bins)*sig_values)
    scale_MCSig = areaBkg/areaSig

    plt.close()
    
    
    ### BACKGROUND COLORS ###
    repeat = [1,4,1,1,15,3]
    colors = [(222/255, 90/255, 106/255),(248/255, 206/255, 104/255),(155/255, 152/255, 204/255),
                (247/255, 138/255, 221/255),(250/255, 202/255, 255/255),(100/255, 192/255, 232/255)]
    bkg_colors = [item for item, count in zip(colors, repeat) for i in range(count)]
    
    
    ### PLOTTING ###
    
    plt.figure(figsize=(8,6))

    plt.hist(bkgMC_bin_centers, weights=bkgMC_counts, bins=bins, label=bkgMC_labels,
             alpha=1., color=bkg_colors, stacked=True, histtype= 'barstacked')
                                                                                                  
    plt.errorbar(data_bin_centers, data_counts, yerr=np.sqrt(data_counts), 
                 fmt='o', ms=4, lw=1, color='black', label='Data', capsize=3)

    plt.hist(SigMC_bin_centers, weights=SigMC_counts, bins=SigMC_bins, #*scale_MCSig
             label='MCSig', color='#00A88F', alpha=1.,  histtype=u'step', linewidth=2)
                                                                                                
    plt.xlabel(variable)
    plt.ylabel('counts')
    
    ST_patch = patches.Patch(color=bkg_colors[0], label='Single Top')
    TT_patch = patches.Patch(color=bkg_colors[1], label='Top Top')
    TTW_patch = patches.Patch(color=bkg_colors[5], label=r'tt$\rightarrow$ W')
    TTZ_patch = patches.Patch(color=bkg_colors[6], label=r'tt$\rightarrow$ Z')
    Diboson_patch = patches.Patch(color=bkg_colors[7], label='Diboson')
    Others_patch = patches.Patch(color=bkg_colors[-1], label='Others')
    MCSig_patch = patches.Patch(color='#00A88F', label='MCSig')
    Data = Line2D([0], [0], marker='o', color='w', label='Data', markerfacecolor='black', markersize=7)

    plt.legend(handles=[ST_patch,TT_patch, TTW_patch, TTZ_patch, Diboson_patch, Others_patch, MCSig_patch, Data])
    plt.tight_layout()
    
    plt.show()

In [ ]:
plot_histogram('inv_m01', (0,200), 30)

## OTHERS

#### Trials pyRDF

In [ ]:
conf = SparkConf().setAppName('appName').setMaster('spark://10.67.22.59:7077')
sc = SparkContext(conf=conf)

In [ ]:
file_list_raw = requests.get(buck, verify=False)
file_list = file_list_raw.text.split('\n')

In [ ]:
for i in range(len(file_list)):
    file_list[i] = 's3' + buck + file_list[i]
    
chain = ROOT.TChain('Events')
for file in file_list[:2]:
    chain.AddFile(file)

In [ ]:
df = PyRDF.RDataFrame(chain, sparkcontext = sc)

#### Distributed PyROOT

In [ ]:
for i in range(len(file_list)):
    file_list[i] = 's3' + buck + file_list[i]
    
chain = ROOT.TChain('Events')
for file in file_list[:1]:
    chain.AddFile(file)

In [ ]:
# Point RDataFrame calls to the Spark specific RDataFrame
RDataFrame = ROOT.RDF.Experimental.Distributed.Spark.RDataFrame

df = RDataFrame(chain, sparkcontext = sc)

#### Stop cluster

In [ ]:
sc.stop()

### To do:

- risorse spark app [here](https://spark.apache.org/docs/latest/configuration.html#memory-management)
- partitions prima o dopo il loading file
- maybe insert: interesting statistics, comparison with root loaders
